In [1]:
import collections
import os
import sys

import tensorflow as tf

In [4]:
def _read_words(filename):
    with tf.gfile.GFile(name=filename, mode='r') as f:
        return f.read().replace('\n','<eos>').split()

In [5]:
train_file = 'ptb.train.txt'

In [6]:
data = _read_words(train_file)

In [8]:
counter = collections.Counter(data)

In [11]:
count_pairs = sorted(counter.items(),key=lambda x: (-x[1],x[0]))

In [13]:
words,_ = list(zip(*count_pairs))

In [15]:
word_to_id = dict(zip(words, range(len(words))))

In [17]:
train_data = [word_to_id[word] for word in data if word in word_to_id]

In [19]:
vocabulary = len(word_to_id)

In [20]:
batch_size=64
num_steps=1000

In [21]:
with tf.name_scope(name='PTBProducer', values=[train_data, batch_size, num_steps]):
    data_len=tf.size(train_data)
    batch_len = data_len//batch_size
    batch_data = tf.reshape(train_data[0:batch_size*batch_len],[batch_size, batch_len])
    epoch_size = (batch_len-1)//num_steps
    assertion = tf.assert_positive(epoch_size, message='epoch_size==0, decrease batch_size or num_steps')
    
    with tf.control_dependencies(control_inputs=[assertion]):
        epoch_size = tf.identity(epoch_size, name='epoch_size')
        
    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.strided_slice(batch_data, [0,i*num_steps], [batch_size, (i+1)*num_steps])
    x.set_shape([batch_size, num_steps])
    
    y = tf.strided_slice(data, [0, i * num_steps + 1],[batch_size, (i + 1) * num_steps + 1])
    y.set_shape([batch_size, num_steps])

TypeError: slice indices must be integers or None or have an __index__ method